In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from tqdm import tqdm
import pickle
import gc
from pathlib import Path
from nuscenes import NuScenes
from scipy.spatial.transform import Rotation as R
from math import cos, sin, pi
from lyft_dataset_sdk.lyftdataset import *
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from second.pytorch.train import build_network, example_convert_to_torch
from second.data.preprocess import merge_second_batch

import torch
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from google.protobuf import text_format
from second.utils import simplevis
from second.pytorch.train import build_network
from second.protos import pipeline_pb2
from second.utils import config_tool

In [20]:
############################################################################## Some hyperparameters we'll need to define for the system
voxel_size = (0.4, 0.4, 1.5)
z_offset = -2.0
bev_shape = (336, 336, 3)

# We scale down each box so they are more separated when projected into our coarse voxel space.
box_scale = 0.8

# If you try to use original LyftDataset Class, you will get missing table error
class LyftTestDataset(LyftDataset):
    """Database class for Lyft Dataset to help query and retrieve information from the database."""

    def __init__(self, data_path: str, json_path: str, verbose: bool = True, map_resolution: float = 0.1):
        """Loads database and creates reverse indexes and shortcuts.
        Args:
            data_path: Path to the tables and data.
            json_path: Path to the folder with json files
            verbose: Whether to print status messages during load.
            map_resolution: Resolution of maps (meters).
        """

        self.data_path = Path(data_path).expanduser().absolute()
        self.json_path = Path(json_path)

        self.table_names = [
            "category",
            "attribute",
            "sensor",
            "calibrated_sensor",
            "ego_pose",
            "log",
            "scene",
            "sample",
            "sample_data",
            "map",
        ]

        start_time = time.time()

        # Explicitly assign tables to help the IDE determine valid class members.
        self.category = self.__load_table__("category")
        self.attribute = self.__load_table__("attribute")
        
        
        self.sensor = self.__load_table__("sensor")
        self.calibrated_sensor = self.__load_table__("calibrated_sensor")
        self.ego_pose = self.__load_table__("ego_pose")
        self.log = self.__load_table__("log")
        self.scene = self.__load_table__("scene")
        self.sample = self.__load_table__("sample")
        self.sample_data = self.__load_table__("sample_data")
        
        self.map = self.__load_table__("map")

        if verbose:
            for table in self.table_names:
                print("{} {},".format(len(getattr(self, table)), table))
            print("Done loading in {:.1f} seconds.\n======".format(time.time() - start_time))

        # Initialize LyftDatasetExplorer class
        self.explorer = LyftDatasetExplorer(self)
        # Make reverse indexes for common lookups.
        self.__make_reverse_index__(verbose)
        
    def __make_reverse_index__(self, verbose: bool) -> None:
        """De-normalizes database to create reverse indices for common cases.
        Args:
            verbose: Whether to print outputs.
        """

        start_time = time.time()
        if verbose:
            print("Reverse indexing ...")

        # Store the mapping from token to table index for each table.
        self._token2ind = dict()
        for table in self.table_names:
            self._token2ind[table] = dict()

            for ind, member in enumerate(getattr(self, table)):
                self._token2ind[table][member["token"]] = ind

        # Decorate (adds short-cut) sample_data with sensor information.
        for record in self.sample_data:
            cs_record = self.get("calibrated_sensor", record["calibrated_sensor_token"])
            sensor_record = self.get("sensor", cs_record["sensor_token"])
            record["sensor_modality"] = sensor_record["modality"]
            record["channel"] = sensor_record["channel"]

        # Reverse-index samples with sample_data and annotations.
        for record in self.sample:
            record["data"] = {}
            record["anns"] = []

        for record in self.sample_data:
            if record["is_key_frame"]:
                sample_record = self.get("sample", record["sample_token"])
                sample_record["data"][record["channel"]] = record["token"]

        if verbose:
            print("Done reverse indexing in {:.1f} seconds.\n======".format(time.time() - start_time))

level5data = LyftTestDataset(data_path='.', json_path='/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/test_root/data', verbose=True)

9 category,
17 attribute,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
1 map,
Done loading in 1.8 seconds.
Reverse indexing ...
Done reverse indexing in 1.9 seconds.


In [27]:
def box_in_image(box, intrinsic, image_size) -> bool:
    """Check if a box is visible inside an image without accounting for occlusions.
    Args:
        box: The box to be checked.
        intrinsic: <float: 3, 3>. Intrinsic camera matrix.
        image_size: (width, height)
        vis_level: One of the enumerations of <BoxVisibility>.
    Returns: True if visibility condition is satisfied.
    """

    corners_3d = box.corners()
    corners_img = view_points(corners_3d, intrinsic, normalize=True)[:2, :]

    visible = np.logical_and(corners_img[0, :] > 0, corners_img[0, :] < image_size[0])
    visible = np.logical_and(visible, corners_img[1, :] < image_size[1])
    visible = np.logical_and(visible, corners_img[1, :] > 0)
    visible = np.logical_and(visible, corners_3d[2, :] > 1)

    in_front = corners_3d[2, :] > 0.1  # True if a corner is at least 0.1 meter in front of the camera.

    return any(visible) and all(in_front)

def viz_pred(sample_token, boxes, all_pred_fn): 

    sample = level5data.get("sample", sample_token)

    sample_camera_token = sample["data"]["CAM_FRONT"]
    camera_data = level5data.get("sample_data", sample_camera_token)
    # camera_filepath = level5data.get_sample_data_path(sample_camera_token)

    ego_pose = level5data.get("ego_pose", camera_data["ego_pose_token"])
    calibrated_sensor = level5data.get("calibrated_sensor", camera_data["calibrated_sensor_token"])
    data_path_, _, camera_intrinsic = level5data.get_sample_data(sample_camera_token)

    data_path = Path('/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/test_root/images/' \
                     + str(data_path_).split('/')[-1])
    data = Image.open(data_path)
#     os.path.join(self.test_data_folder,f"{sample_token}_input.png")
    _, axis = plt.subplots(1, 1, figsize=(9, 9))
    
    for i,box in enumerate(boxes):

        # Move box to ego vehicle coord system
        box.translate(-np.array(ego_pose["translation"]))
        box.rotate(Quaternion(ego_pose["rotation"]).inverse)

        # Move box to sensor coord system
        box.translate(-np.array(calibrated_sensor["translation"]))
        box.rotate(Quaternion(calibrated_sensor["rotation"]).inverse)

        if box_in_image(box,camera_intrinsic,np.array(data).shape):            
            box.render(axis,camera_intrinsic,normalize=True)

    axis.imshow(data)
    all_pred_fn.append(f'./cam_viz/cam_preds_{sample_token}.jpg')
    plt.savefig(all_pred_fn[-1])
    plt.close()
    
    return all_pred_fn

In [30]:
def thresholded_pred(pred, threshold):
    try:
        box3d = pred["box3d_lidar"].detach().cpu().numpy()
        scores = pred["scores"].detach().cpu().numpy()
        labels = pred["label_preds"].detach().cpu().numpy()
    except:
        box3d = pred["box3d_lidar"]
        scores = pred["scores"]
        labels = pred["label_preds"]
    idx = np.where(scores > threshold)[0]
    # filter low score ones
    box3d = box3d[idx, :]
    # label is one-dim
    labels = np.take(labels, idx)
    scores = np.take(scores, idx)
    pred['box3d_lidar'] = box3d
    pred['scores'] = scores
    pred['label_preds'] = labels
    return pred

def to_glb(box, info):
    # lidar -> ego -> global
    # info should belong to exact same element in `gt` dict
    box.rotate(Quaternion(info['lidar2ego_rotation']))
    box.translate(np.array(info['lidar2ego_translation']))
    box.rotate(Quaternion(info['ego2global_rotation']))
    box.translate(np.array(info['ego2global_translation']))
    return box

def get_pred_str(pred, sample_token, classes, vid_count, processed_samples, all_pred_fn):
    boxes_lidar = pred["box3d_lidar"]
    boxes_class = pred["label_preds"]
    scores = pred['scores']
    preds_classes = [classes[x] for x in boxes_class]
    box_centers = boxes_lidar[:, :3]
    box_yaws = boxes_lidar[:, -1]
    box_wlh = boxes_lidar[:, 3:6]
    info = token2info[sample_token] # a `sample` token
    boxes = []
    pred_str = ''
    max_frames = 128
    for idx in range(len(boxes_lidar)):
        translation = box_centers[idx]
        yaw = - box_yaws[idx] - pi/2
        size = box_wlh[idx]
        name = preds_classes[idx]
        detection_score = scores[idx]
        quat = Quaternion(scalar=np.cos(yaw / 2), vector=[0, 0, np.sin(yaw / 2)])
        box = Box(
            center=box_centers[idx],
            size=size,
            orientation=quat,
            score=detection_score,
            name=name,
            token=sample_token
        )
        boxes.append(box)
        box = to_glb(box, info)
        pred =  str(box.score) + ' ' + str(box.center[0])  + ' ' \
                + str(box.center[1]) + ' '  + str(box.center[2]) + ' '  \
                + str(box.wlh[0]) + ' ' + str(box.wlh[1]) + ' '  +  \
                str(box.wlh[2]) + ' ' + str(box.orientation.yaw_pitch_roll[0]) \
                + ' ' + str(name) + ' '
        pred_str += pred
        
    if vid_count < 1:
        all_pred_fn = viz_pred(sample_token, boxes, all_pred_fn)
        if processed_samples == max_frames:
            os.makedirs('./cam_viz',exist_ok=True)
            processed_samples = 0
            vid_count += 1        
            new_clip = ImageSequenceClip(all_pred_fn,fps=5)
            all_pred_fn = []
            new_clip.write_videofile(f"model_preds_{vid_count}.mp4") 
            shutil.rmtree('./cam_viz')
            del new_clip
            gc.collect()
            os.makedirs('./cam_viz',exist_ok=True)
    
    return pred_str.strip(), vid_count, all_pred_fn

In [13]:
config_path = "/media/jionie/my_disk/Kaggle/Lyft/codes/second/second/configs/nuscenes/all.pp.lowa.config"
info_path = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/test_root/infos_test.pkl'
root_path = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/test_root/'
df = pd.read_csv('/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/test_root/sample_submission.csv')
ckpt_path = "/media/jionie/my_disk/Kaggle/Lyft/model/second/voxelnet-235437.tckpt"
config = pipeline_pb2.TrainEvalPipelineConfig()

with open(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, config)
input_cfg = config.eval_input_reader
model_cfg = config.model.second
# config_tool.change_detection_range_v2(model_cfg, [-50, -50, 50, 50])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open(info_path, 'rb') as f:
    infos = pickle.load(f)

token2info = {}
for info in infos['infos']:
    token2info[info['token']] = info

net = build_network(config.model.second).to(device).float().eval()
net.load_state_dict(torch.load(ckpt_path))
eval_input_cfg = config.eval_input_reader
eval_input_cfg.dataset.kitti_root_path = root_path
eval_input_cfg.dataset.kitti_info_path = info_path
dataset = input_reader_builder.build(
    eval_input_cfg,
    config.model.second,
    training=False,
    voxel_generator=net.voxel_generator,
    target_assigner=net.target_assigner)#.dataset

batch_size = 2
num_workers = 2

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size, # only support multi-gpu train
    shuffle=False,
    num_workers=num_workers,
    pin_memory=False,
    collate_fn=merge_second_batch)

target_assigner = net.target_assigner
voxel_generator = net.voxel_generator
classes = target_assigner.classes

token2predstr = {}
detections = []
#tokens = []
tk0 = tqdm(dataloader, total=len(dataloader))
for idx, examples in enumerate(tk0):
    try:
        example_torch = example_convert_to_torch(examples, device=device)
        detections += net(example_torch)
        #tokens += examples['metadata']
    except Exception as e:
        print(e)
        import pdb; pdb.set_trace()

100%|██████████| 13734/13734 [48:33<00:00,  5.96it/s] 


In [33]:
os.makedirs('./cam_viz',exist_ok=True)

from moviepy.editor import ImageSequenceClip 
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision
import shutil

threshold = 0.2
vid_count = 0
processed_samples = 0
all_pred_fn = []
for idx, pred in enumerate(tqdm(detections)):
    processed_samples += 1
    pred = thresholded_pred(pred, threshold)
    #token = tokens[idx]['token']
    token = pred['metadata']['token']
    pred_str, vid_count, all_pred_fn = \
        get_pred_str(pred, token, classes, vid_count, processed_samples, all_pred_fn)
    index = df[df['Id'] == token].index[0]
    df.loc[index, 'PredictionString'] = pred_str

df.to_csv(f'final.csv', index=False)



  0%|          | 0/27468 [00:00<?, ?it/s]

  0%|          | 1/27468 [00:00<1:16:03,  6.02it/s]

  0%|          | 2/27468 [00:00<1:11:17,  6.42it/s]

  0%|          | 3/27468 [00:00<1:07:23,  6.79it/s]

  0%|          | 4/27468 [00:00<1:05:09,  7.02it/s]

  0%|          | 5/27468 [00:00<1:04:52,  7.06it/s]

  0%|          | 6/27468 [00:00<1:05:15,  7.01it/s]

  0%|          | 7/27468 [00:00<1:05:16,  7.01it/s]

  0%|          | 8/27468 [00:01<1:04:13,  7.13it/s]

  0%|          | 9/27468 [00:01<1:02:55,  7.27it/s]

  0%|          | 10/27468 [00:01<1:02:01,  7.38it/s]

  0%|          | 11/27468 [00:01<1:00:41,  7.54it/s]

  0%|          | 12/27468 [00:01<1:00:37,  7.55it/s]

  0%|          | 13/27468 [00:01<1:01:58,  7.38it/s]

  0%|          | 14/27468 [00:01<1:01:36,  7.43it/s]

  0%|          | 15/27468 [00:02<1:00:53,  7.51it/s]

  0%|          | 16/27468 [00:02<1:01:07,  7.49it/s]

  0%|          | 17/27468 [00:02<1:00:36,  7.55it/s]

  0%|          | 18/27468 [00:02<1:00:44,  7.5

t:   0%|          | 0/128 [00:00<?, ?it/s, now=None]

Moviepy - Building video model_preds_1.mp4.
Moviepy - Writing video model_preds_1.mp4






t:  10%|█         | 13/128 [00:00<00:00, 126.65it/s, now=None]


t:  19%|█▉        | 24/128 [00:00<00:00, 120.79it/s, now=None]


t:  27%|██▋       | 35/128 [00:00<00:00, 116.81it/s, now=None]


t:  36%|███▌      | 46/128 [00:00<00:00, 111.30it/s, now=None]


t:  45%|████▍     | 57/128 [00:00<00:00, 109.19it/s, now=None]


t:  52%|█████▏    | 66/128 [00:00<00:00, 102.57it/s, now=None]


t:  59%|█████▊    | 75/128 [00:00<00:00, 96.05it/s, now=None] 


t:  66%|██████▋   | 85/128 [00:00<00:00, 95.60it/s, now=None]


t:  74%|███████▍  | 95/128 [00:00<00:00, 94.54it/s, now=None]


t:  82%|████████▏ | 105/128 [00:01<00:00, 95.38it/s, now=None]


t:  90%|████████▉ | 115/128 [00:01<00:00, 95.25it/s, now=None]


t:  98%|█████████▊| 125/128 [00:01<00:00, 94.07it/s, now=None]


                                                              

                                               
  0%|          | 127/27468 [07:00<2:10:12,  3.50it/s]

  0%|          | 127/27468 [00:30<1:38:25,  4.63it/s

Moviepy - Done !
Moviepy - video ready model_preds_1.mp4


...

 69%|██████▉   | 18991/27468 [03:04<00:41, 204.20it/s]

 69%|██████▉   | 19012/27468 [03:05<00:41, 204.53it/s]

 69%|██████▉   | 19034/27468 [03:05<00:40, 207.36it/s]

 69%|██████▉   | 19055/27468 [03:05<00:40, 207.57it/s]

 69%|██████▉   | 19076/27468 [03:05<00:44, 186.92it/s]

 70%|██████▉   | 19096/27468 [03:05<00:52, 159.19it/s]

 70%|██████▉   | 19113/27468 [03:05<00:54, 154.32it/s]

 70%|██████▉   | 19136/27468 [03:05<00:49, 169.90it/s]

 70%|██████▉   | 19157/27468 [03:05<00:46, 179.72it/s]

 70%|██████▉   | 19176/27468 [03:06<00:49, 168.47it/s]

 70%|██████▉   | 19194/27468 [03:06<00:48, 171.12it/s]

 70%|██████▉   | 19212/27468 [03:06<00:52, 158.73it/s]

 70%|███████   | 19230/27468 [03:06<00:50, 162.63it/s]

 70%|███████   | 19247/27468 [03:06<00:51, 158.51it/s]

 70%|███████   | 19264/27468 [03:06<00:52, 156.40it/s]

 70%|███████   | 19281/27468 [03:06<00:51, 158.87it/s]

 70%|███████   | 19302/27468 [03:06<00:48, 169.90it/s]

 70%|███████   | 19320/27468 [03:06<00:47, 170.6

 77%|███████▋  | 21060/27468 [03:20<00:48, 132.23it/s]

 77%|███████▋  | 21074/27468 [03:20<00:47, 133.32it/s]

 77%|███████▋  | 21088/27468 [03:20<00:48, 132.38it/s]

 77%|███████▋  | 21103/27468 [03:20<00:46, 136.28it/s]

 77%|███████▋  | 21121/27468 [03:20<00:43, 145.70it/s]

 77%|███████▋  | 21139/27468 [03:20<00:41, 152.78it/s]

 77%|███████▋  | 21155/27468 [03:20<00:41, 153.10it/s]

 77%|███████▋  | 21172/27468 [03:20<00:40, 156.53it/s]

 77%|███████▋  | 21188/27468 [03:20<00:46, 136.46it/s]

 77%|███████▋  | 21203/27468 [03:21<00:51, 120.97it/s]

 77%|███████▋  | 21216/27468 [03:21<00:53, 117.39it/s]

 77%|███████▋  | 21229/27468 [03:21<00:52, 119.78it/s]

 77%|███████▋  | 21243/27468 [03:21<00:50, 123.25it/s]

 77%|███████▋  | 21260/27468 [03:21<00:46, 132.79it/s]

 77%|███████▋  | 21278/27468 [03:21<00:43, 143.43it/s]

 78%|███████▊  | 21296/27468 [03:21<00:40, 151.26it/s]

 78%|███████▊  | 21312/27468 [03:21<00:40, 152.04it/s]

 78%|███████▊  | 21331/27468 [03:21<00:38, 161.3

 84%|████████▍ | 23168/27468 [03:35<00:36, 117.95it/s]

 84%|████████▍ | 23181/27468 [03:35<00:37, 114.93it/s]

 84%|████████▍ | 23193/27468 [03:35<00:37, 113.52it/s]

 84%|████████▍ | 23205/27468 [03:35<00:37, 113.35it/s]

 85%|████████▍ | 23217/27468 [03:35<00:38, 111.23it/s]

 85%|████████▍ | 23229/27468 [03:35<00:40, 105.73it/s]

 85%|████████▍ | 23241/27468 [03:35<00:39, 108.13it/s]

 85%|████████▍ | 23252/27468 [03:35<00:40, 104.60it/s]

 85%|████████▍ | 23263/27468 [03:36<00:41, 102.30it/s]

 85%|████████▍ | 23275/27468 [03:36<00:39, 105.91it/s]

 85%|████████▍ | 23289/27468 [03:36<00:36, 114.01it/s]

 85%|████████▍ | 23307/27468 [03:36<00:32, 126.64it/s]

 85%|████████▍ | 23328/27468 [03:36<00:28, 143.13it/s]

 85%|████████▌ | 23349/27468 [03:36<00:26, 157.56it/s]

 85%|████████▌ | 23371/27468 [03:36<00:23, 171.14it/s]

 85%|████████▌ | 23392/27468 [03:36<00:22, 181.04it/s]

 85%|████████▌ | 23415/27468 [03:36<00:21, 192.53it/s]

 85%|████████▌ | 23437/27468 [03:36<00:20, 196.7

 91%|█████████▏| 25128/27468 [03:50<00:21, 108.51it/s]

 92%|█████████▏| 25140/27468 [03:50<00:22, 103.12it/s]

 92%|█████████▏| 25151/27468 [03:50<00:22, 105.06it/s]

 92%|█████████▏| 25166/27468 [03:50<00:20, 115.10it/s]

 92%|█████████▏| 25185/27468 [03:50<00:17, 129.84it/s]

 92%|█████████▏| 25203/27468 [03:50<00:16, 139.93it/s]

 92%|█████████▏| 25218/27468 [03:50<00:17, 130.43it/s]

 92%|█████████▏| 25232/27468 [03:51<00:17, 124.91it/s]

 92%|█████████▏| 25246/27468 [03:51<00:18, 118.95it/s]

 92%|█████████▏| 25259/27468 [03:51<00:19, 113.07it/s]

 92%|█████████▏| 25271/27468 [03:51<00:19, 111.39it/s]

 92%|█████████▏| 25283/27468 [03:51<00:19, 111.73it/s]

 92%|█████████▏| 25295/27468 [03:51<00:19, 108.91it/s]

 92%|█████████▏| 25307/27468 [03:51<00:20, 106.92it/s]

 92%|█████████▏| 25318/27468 [03:51<00:21, 102.18it/s]

 92%|█████████▏| 25329/27468 [03:51<00:21, 99.44it/s] 

 92%|█████████▏| 25340/27468 [03:52<00:20, 101.64it/s]

 92%|█████████▏| 25354/27468 [03:52<00:19, 109.9

 99%|█████████▊| 27057/27468 [04:05<00:04, 100.39it/s]

 99%|█████████▊| 27068/27468 [04:05<00:03, 100.01it/s]

 99%|█████████▊| 27079/27468 [04:05<00:03, 99.20it/s] 

 99%|█████████▊| 27089/27468 [04:05<00:03, 98.13it/s]

 99%|█████████▊| 27101/27468 [04:05<00:03, 101.85it/s]

 99%|█████████▊| 27114/27468 [04:05<00:03, 108.41it/s]

 99%|█████████▉| 27131/27468 [04:05<00:02, 121.56it/s]

 99%|█████████▉| 27150/27468 [04:06<00:02, 135.39it/s]

 99%|█████████▉| 27170/27468 [04:06<00:02, 148.42it/s]

 99%|█████████▉| 27190/27468 [04:06<00:01, 159.28it/s]

 99%|█████████▉| 27211/27468 [04:06<00:01, 170.51it/s]

 99%|█████████▉| 27229/27468 [04:06<00:01, 157.24it/s]

 99%|█████████▉| 27246/27468 [04:06<00:01, 144.31it/s]

 99%|█████████▉| 27262/27468 [04:06<00:01, 137.58it/s]

 99%|█████████▉| 27277/27468 [04:06<00:01, 131.68it/s]

 99%|█████████▉| 27291/27468 [04:07<00:01, 125.96it/s]

 99%|█████████▉| 27304/27468 [04:07<00:01, 122.93it/s]

 99%|█████████▉| 27323/27468 [04:07<00:01, 136.56